<a href="https://colab.research.google.com/github/MustafizSaadi/DMLab/blob/main/K_medoids_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import gzip
from io import StringIO
import re
import sys
import math
import statistics 
import random
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

In [ ]:
# cd /content/drive/My\ Drive

In [ ]:
def load_dataset(file_name, threshold):
  attribute_dictionary = dict()
  max_dictionary = dict()
  min_dictionary = dict()
  category_values = dict()
  df = pd.read_csv(file_name, header = None)

  df = df.astype("float32")
  print(df.dtypes)

  full_table = df.to_numpy()
  print(full_table)

  for i in range(full_table.shape[1]):
    print(full_table[:,i])
    elements = np.unique(full_table[:,i])
    ratio =  len(elements)/full_table.shape[0]

    if ratio >= threshold:
      attribute_dictionary[i] = "numeric"
      max_dictionary[i] = max(elements)
      min_dictionary[i] = min(elements)
    else:
      attribute_dictionary[i] = "category"
      category_values[i] = elements
  
  print(attribute_dictionary)
  
  return attribute_dictionary, max_dictionary, min_dictionary, category_values, full_table

In [ ]:
def replace_nan_with_most_frequent_category(df, column):
  most_frequent_category = df.iloc[:,column].mode()[0]
  print(most_frequent_category)
  df.iloc[:,column].fillna(most_frequent_category, inplace = True)
  print(df.iloc[:,column])
  print("within replace")
  return df

In [ ]:
def load_dataset_sponge(file_name, threshold):
  attribute_dictionary = dict()
  max_dictionary = dict()
  min_dictionary = dict()
  category_values = dict()
  df = pd.read_csv(file_name, header = None)
  
  df = df.drop(df.columns[0], axis = 1)
  df = df.replace('?', np.nan)
  df = df.astype("object")
  print("dataframe type", df.dtypes)
  print("start")
  
  for i in range(df.shape[1]):
    if df.iloc[:,i].isnull().values.any():
      print(1)
      df = replace_nan_with_most_frequent_category(df, i)
    # print(df.columns[i])

  print(df.shape)
  full_table = df.to_numpy()
  print(full_table.shape[1])

  for i in range(full_table.shape[1]):
    print(full_table[:,i])
    elements = np.unique(full_table[:,i])
    ratio =  len(elements)/full_table.shape[0]

    if ratio >= threshold:
      attribute_dictionary[i] = "numeric"
      max_dictionary[i] = max(elements)
      min_dictionary[i] = min(elements)
    else:
      attribute_dictionary[i] = "category"
      category_values[i] = elements
  
  print(attribute_dictionary)
  
  return attribute_dictionary,  max_dictionary, min_dictionary, category_values, full_table

In [ ]:
def distance(attribute_dictionary,  max_dictionary, min_dictionary, data, mean):
  dist = 0
  col = 0
  for i,j in zip(data, mean):
    if attribute_dictionary[col] == "numeric":
      dist += abs(float(i)-float(j))/(max_dictionary[col] - min_dictionary[col])
    elif attribute_dictionary[col] == "category":
      if i != j:
        dist += 1
    col += 1
  # print("distance", dist/col)
  return dist/col

In [ ]:
def find_closest_median(attribute_dictionary,  max_dictionary, min_dictionary, data, database, means):
  idx = -1
  min = sys.maxsize

  for mean_idx in means:
    # mean = means[i]
    dist = distance(attribute_dictionary,  max_dictionary, min_dictionary, data, database[mean_idx,:])

    if dist < min:
      min = dist
      idx = mean_idx

  return min, idx

In [ ]:
def Kmedoid(attribute_dictionary,  max_dictionary, min_dictionary, database, means):
  clusters = dict()
  m = 0
  while m < 20:
    cur_cost = 0
    flag = False

    print("means", means)

    for i in range(len(means)):
      clusters[i] = []
  
    for data in database:
  
      min, idx = find_closest_median(attribute_dictionary,  max_dictionary, min_dictionary, data, database, means)

      cur_cost += min
      clusters[means.index(idx)].append(data)


    l = 0
    while l < 0.2*(database.shape[0]):
      random_mean = random.randint(0, len(means)-1)

      new_means = []
      checked = []
      for mean_idx in means:
        new_means.append(mean_idx)
        checked.append(mean_idx)

      while True:
        if len(checked) == database.shape[0]:
          break

        non_representative_point = random.randint(0, database.shape[0]-1)

        if non_representative_point not in checked:
          # checked.append(non_representative_point)

          new_means[random_mean] = non_representative_point

          new_cost = 0

          for data in database:
        
            min, _ = find_closest_median(attribute_dictionary,  max_dictionary, min_dictionary, data, database, new_means)

            new_cost += min
          
          if new_cost < cur_cost:
            cur_cost = new_cost
            flag = True
            means[random_mean] = new_means[random_mean]
            print(means)
          break
      l += 1

    if not flag:
      break

    m += 1

  return cur_cost, means, clusters
          


In [ ]:
def calculate_pairwise_distance(sillhoutte_dataset, attribute_dictionary, max_dictionary, min_dictionary):
  row = sillhoutte_dataset.shape[0]
  pairwise_distance = np.zeros((row,row))

  for i in range(row):
    for j in range(i+1, row):
      pairwise_distance[i][j] = distance(attribute_dictionary, max_dictionary, min_dictionary, sillhoutte_dataset[i], sillhoutte_dataset[j])
      pairwise_distance[j][i] = pairwise_distance[i][j]

  return pairwise_distance
  

In [ ]:
threshold = 0.05
# attribute_dictionary,  max_dictionary, min_dictionary, category_values, database = load_dataset('unlabeled data', threshold)
# attribute_dictionary,  max_dictionary, min_dictionary, category_values, database = load_dataset_sponge('sponge.csv', threshold)
# attribute_dictionary, max_dictionary, min_dictionary, category_values, database = load_dataset('QCM3.csv', threshold)
# attribute_dictionary, max_dictionary, min_dictionary, category_values, database = load_dataset('buddymove_holidayiq.csv', threshold)
# attribute_dictionary, max_dictionary, min_dictionary, category_values, database = load_dataset('tripadvisor_review.csv', threshold)
attribute_dictionary, max_dictionary, min_dictionary, category_values, database = load_dataset('Sales_Transactions_Dataset_Weekly.csv', threshold)
# print(category_values)
max_k = np.linspace(1, 20, num = 20)
# np.linspace(1, 10, num = 10)
# X = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
# Y = [143,116,96,91,86,82,81,79,77,76,74,73,72,72,70,70,69,68,67]
# np.zeros((k, database.shape[1]))
iterations = 3
for k in max_k:
  print("******* ", k)
  best_means = []
  best_err = sys.maxsize
  best_cluster = dict()
  # min_err = sys.maxsize
  for it in range(iterations):
    means = []
    err = 0
    clusters = dict()
    np.random.shuffle(database)
    for i in range(int(k)):
      # temp = []
      # for j in range(database.shape[1]):
      #   temp.append(database[i][j])
      means.append(i)
    err, means, clusters = Kmedoid(attribute_dictionary,  max_dictionary, min_dictionary, database, means)
    if err < best_err:
      best_means = means
      best_err = err
      best_cluster = clusters
    
  print("means", best_means)
  print(k, best_err)
  X.append(k)
  Y.append(best_err)
means = np.array(means)

plt.plot(X, Y)
plt.show()

0     float32
1     float32
2     float32
3     float32
4     float32
5     float32
6     float32
7     float32
8     float32
9     float32
10    float32
11    float32
12    float32
13    float32
14    float32
15    float32
16    float32
17    float32
18    float32
19    float32
20    float32
21    float32
22    float32
23    float32
24    float32
25    float32
26    float32
27    float32
28    float32
29    float32
30    float32
31    float32
32    float32
33    float32
34    float32
35    float32
36    float32
37    float32
38    float32
39    float32
40    float32
41    float32
42    float32
43    float32
44    float32
45    float32
46    float32
47    float32
48    float32
49    float32
50    float32
51    float32
dtype: object
[[11. 12. 10. ...  6.  5. 10.]
 [ 7.  6.  3. ...  1.  6.  0.]
 [ 7. 11.  8. ...  8.  8.  7.]
 ...
 [ 1.  0.  0. ...  0.  4.  3.]
 [ 0.  0.  0. ...  0.  2.  0.]
 [ 0.  1.  0. ...  0.  0.  1.]]
[11.  7.  7. 12.  8.  3.  4.  8. 14. 22. 15.  3. 12. 14. 19. 30. 4

NameError: ignored

In [ ]:
best_means = []
best_err = sys.maxsize
best_cluster = dict()
# min_err = sys.maxsize
k = 6
for it in range(iterations):
  means = []
  err = 0
  clusters = dict()
  np.random.shuffle(database)
  for i in range(k):
    # temp = []
    # for j in range(database.shape[1]):
    #   temp.append(database[i][j])
    means.append(i)
  err, means, clusters = Kmedoid(attribute_dictionary,  max_dictionary, min_dictionary, database, means)
  if err < best_err:
    best_means = means
    best_err = err
    best_cluster = clusters
    sillhoutte_dataset = []
    sillhoutte_label = []
    for cluster in best_cluster:
      for data in best_cluster[cluster]:
        sillhoutte_dataset.append(data)
        sillhoutte_label.append(cluster)

    sillhoutte_dataset = np.array(sillhoutte_dataset)

    pairwise_distance = calculate_pairwise_distance(sillhoutte_dataset, attribute_dictionary, max_dictionary, min_dictionary)

    # print(pairwise_distance)

    silhouette_avg = silhouette_score(pairwise_distance, sillhoutte_label, metric="precomputed")
    print("Sillhoutte score ->", silhouette_avg)
    print("Best error ->", best_err)

In [ ]:
print(best_means)

In [ ]:
for ind in best_cluster:
  print(ind, len(best_cluster[ind]))

In [ ]:
l = [0,1,2,'hi']
arr = np.array(l)
arr[0] = float(arr[0]) + 11
print(arr)

In [ ]:
print(best_err)